In [ ]:
%pip install -qU langchain-aws boto3 langchain

In [1]:
import boto3

session = boto3.Session()
textract = session.client(
    service_name='textract', 
    region_name='us-east-1')

In [ ]:
# Textract
from langchain.document_loaders import AmazonTextractPDFLoader

file_path = "s3://bucket-ufms-document/docs/INSTRUCAO NORMATIVA (PROGRAD_RTR) n 4, de 20-06-2023..pdf"

loader = AmazonTextractPDFLoader(file_path, client=textract)
docs = loader.load()

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(model="anthropic.claude-3-5-sonnet-20240620-v1:0",
    beta_use_converse_api=True)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("user", "Write a concise summary of the following:\n\n{context}")]
)

# Instantiate the chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke the chain
result = chain.invoke({"context": docs})
print(result)


In [ ]:
for token in chain.stream({"context": docs}):
    print(token, end="|")